In [14]:
from research import researcher

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
from torch import nn
import torch

In [16]:
data = load_iris()

In [17]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [18]:
data['feature_names'] , data['target_names']

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

data = {
    "X_train": X_train,
    "X_test": X_test,
    "y_train": y_train,
    "y_test": y_test
}

In [20]:
import torch
import torch.nn as nn

class regression_model(nn.Module):
    def __init__(self, input_size, output_size):
        """
        input_size: number of input features
        output_size: number of classes
        hidden_size: number of neurons in hidden layers
        num_hidden_layers: number of hidden layers
        dropout: dropout probability
        """
        super().__init__()

        layers = []  # flatten input

        # 1 layer
        layers.append(nn.Linear(input_size, output_size))
        layers.append(nn.Softmax(dim=1))

        # Combine all layers into a sequential module
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)



In [21]:
model = regression_model(input_size=4, output_size=3)

In [ ]:
config = {
    "hyperparameter": {
        "range": {
            "batch_size": [16, 32, 16],
        },
        "choice": {
            "learning_rate": [0.001, 0.01]
        },
        "fixed": {
            "epochs": 50,
            "loss_fn": nn.CrossEntropyLoss(),
            "optimizer": torch.optim.Adam
        }
    }
}

In [23]:
def train_loop(self):

    model = self.model
    loss_fn = self.loss_fn
    lr = self.learning_rate
    optimizer = self.optimizer(model.parameters(), lr=lr)
    epochs = self.epochs
    batch_size = self.batch_size
    accuracy_fn = self.accuracy_fn
    info = None

    data = self.data

    X_train = torch.tensor(data.X_train, dtype=torch.float32)
    y_train = torch.tensor(data.y_train, dtype=torch.float32)
    X_test = torch.tensor(data.X_test, dtype=torch.float32)
    y_test = torch.tensor(data.y_test, dtype=torch.float32)

    total_loss_train = 0
    total_loss_test = 0
    total_acc_train = 0
    total_acc_test = 0

    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    for epoch in range(epochs):
        model.train()

        total_loss_train = 0
        total_loss_test = 0
        total_acc_train = 0
        total_acc_test = 0
        
        for bt , (X_train, y_train) in enumerate(train_loader):

            y_train_logit = model(X_train)
            y_train_pred = torch.round(y_train_logit)

            loss = loss_fn(y_train_logit, y_train.long())
            total_loss_train += loss.item() * X_train.size(0)
            total_acc_train += accuracy_fn(y_train, y_train_pred) * X_train.size(0)

            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()

        model.eval()

        with torch.inference_mode():
            model.eval()
            for bt , (X_test, y_test) in enumerate(test_loader):

                y_test_logit = model(X_test)
                y_test_pred = torch.round(y_test_logit)

                loss_test = loss_fn(y_test_logit, y_test.long())
                total_loss_test += loss_test.item() * X_test.size(0)
                total_acc_test += accuracy_fn(y_test, y_test_pred) * X_test.size(0)

        avg_loss_train = total_loss_train / len(train_loader.dataset)
        avg_loss_test = total_loss_test / len(test_loader.dataset)
        avg_acc_train = total_acc_train / len(train_loader.dataset)
        avg_acc_test = total_acc_test / len(test_loader.dataset)

        if epoch % (epochs/10) == 0:
            print(f"Epoch {epoch+1} | Train Loss: {avg_loss_train:.4f} | Test Loss: {avg_loss_test:.4f} | Train Acc: {avg_acc_train:.2f} | Test Acc: {avg_acc_test:.2f}")

        info = {
            "batch_size": batch_size,
            "learning_rate": lr
        }
        self.bestModel(avg_loss_train, avg_loss_test, model, epoch, info)

    return {
        "avg_loss_train": avg_loss_train,
        "avg_loss_test": avg_loss_test,
        "model": model,
        "epoch": epoch
    }

In [24]:
argument = {
    "data": data,
    "model": model,
    "train_loop": train_loop,
    "config": config,
    "grid_search": True,
    "research_type": 1
}
model_researcher = researcher(**argument)

Hyperparameter Researcher initialized.
  batch_size: 16
  learning_rate: 0.001
  epochs: 50
  loss_fn: CrossEntropyLoss()
  optimizer: <class 'torch.optim.adam.Adam'>


In [25]:
model_researcher.startResearch()

Starting hyperparameter research... with train type: <bound method researcher.startGridSearchHyperparameter of <research.researcher object at 0x000002296AC68A10>>
Training with hyperparameters: <research.obj object at 0x000002296AB68C50>
Prepared training data and hyperparameters for training loop.
  batch_size: 16
  learning_rate: 0.001
  epochs: 50
  loss_fn: CrossEntropyLoss()
  optimizer: <class 'torch.optim.adam.Adam'>
Epoch 1 | Train Loss: 1.1839 | Test Loss: 1.1530 | Train Acc: 32.50 | Test Acc: 36.67
Epoch 6 | Train Loss: 1.1615 | Test Loss: 1.1313 | Train Acc: 32.50 | Test Acc: 36.67
Epoch 11 | Train Loss: 1.1371 | Test Loss: 1.1126 | Train Acc: 32.50 | Test Acc: 36.67
Epoch 16 | Train Loss: 1.1170 | Test Loss: 1.0983 | Train Acc: 32.50 | Test Acc: 36.67
Epoch 21 | Train Loss: 1.1011 | Test Loss: 1.0868 | Train Acc: 65.00 | Test Acc: 70.00
Epoch 26 | Train Loss: 1.0861 | Test Loss: 1.0718 | Train Acc: 61.67 | Test Acc: 66.67
Epoch 31 | Train Loss: 1.0676 | Test Loss: 1.0511 | 

In [26]:
model_researcher.print_info()

Best Model Info:
Epoch: 49
Train Loss: 0.7261425813039144
Test Loss: 0.7127012292544047
Additional Info: {'batch_size': 16, 'learning_rate': 0.009999999999999998}
